In [0]:
#Define service principals
service_credential = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-pw')
service_application_id = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-app-id')
directory_id = "5f9dc6bd-f38a-454a-864c-c803691193c5"
storage_account = 'sa8451entlakegrnprd'

In [0]:
#Set configurations
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", service_application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
#Define service principals
service_credential = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-pw')
service_application_id = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-app-id')
directory_id = "5f9dc6bd-f38a-454a-864c-c803691193c5"
storage_account = 'sa8451dbxadhocprd'

In [0]:
#Set configurations
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", service_application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
#Define service principals
service_credential = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-pw')
service_application_id = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-app-id')
directory_id = "5f9dc6bd-f38a-454a-864c-c803691193c5"
storage_account = 'sa8451posprd'

In [0]:
#Set configurations
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", service_application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
# IMPORT PACKAGES
from effodata import ACDS, golden_rules, Joiner, Sifter, Equality, sifter, join_on, joiner
from kpi_metrics import (
     KPI,
     AliasMetric,
     CustomMetric,
     AliasGroupby,
     Rollup,
     Cube,
     available_metrics,
     get_metrics
)

from datetime import datetime
from dateutil.relativedelta import relativedelta

# minus 1 year
today_year_ago = (datetime.today() - relativedelta(years=1)).strftime('%Y-%m-%d')

today = datetime.today().strftime('%Y-%m-%d')
yesterday = (datetime.today() - relativedelta(days=1)).strftime('%Y%m%d')

# SPARK SETUP
acds = ACDS(use_sample_mart=False)
kpi = KPI(use_sample_mart=False)

# Pull acds transactions over the past year
acds_pyear_transactions = acds.get_transactions(today_year_ago, today, apply_golden_rules=golden_rules(['customer_exclusions']))
pyear_purchased_upcs = acds_pyear_transactions.select(acds_pyear_transactions.gtin_no).distinct()
pyear_purchased_upcs = pyear_purchased_upcs.select(pyear_purchased_upcs.gtin_no.alias('upc'))

def get_latest_modified_directory(pDirectory):
    """
    get_latest_modified_file_from_directory:
        For a given path to a directory in the data lake, return the directory that was last modified. 
        Input path format expectation: 'abfss://x@sa8451x.dfs.core.windows.net/
    """
    #Set to get a list of all folders in this directory and the last modified date time of each
    vDirectoryContentsList = list(dbutils.fs.ls(pDirectory))

    #Convert the list returned from get_dir_content into a dataframe so we can manipulate the data easily. Provide it with column headings. 
    #You can alternatively sort the list by LastModifiedDateTime and get the top record as well. 
    df = spark.createDataFrame(vDirectoryContentsList,['FullFilePath', 'LastModifiedDateTime'])

    #Get the latest modified date time scalar value
    maxLatestModifiedDateTime = df.agg({"LastModifiedDateTime": "max"}).collect()[0][0]

    #Filter the data frame to the record with the latest modified date time value retrieved
    df_filtered = df.filter(df.LastModifiedDateTime == maxLatestModifiedDateTime)
    
    #return the file name that was last modifed in the given directory
    return df_filtered.first()['FullFilePath']

In [0]:
#To do:  look for latest pinto and pim files
pinto_path = get_latest_modified_directory('abfss://data@sa8451entlakegrnprd.dfs.core.windows.net/source/third_party/prd/pinto/')
pinto_prods = spark.read.parquet(pinto_path)
pim_path = get_latest_modified_directory("abfss://pim@sa8451posprd.dfs.core.windows.net/pim_core/by_cycle/")
pim_core = spark.read.parquet(pim_path)
search_path = "abfss://personloyalty@sa8451dbxadhocprd.dfs.core.windows.net/relevancy/e451_query2concept2vec/20221007/query2concept2vec.parquet"
search_emb = spark.read.parquet(search_path)

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.functions import collect_set, substring_index, concat_ws, concat, split, regexp_replace, size, expr

#Build sentence with upc, description and diet related meta data from PIM data source
pim_diet = pim_core.select(f.col("upc_key"), f.col("krogerOwnedEcommerceDescription"), f.col("gtinName"), f.col("diets.*"))
pim_diets = pim_diet.withColumn('diet', concat_ws(', ', f.col('AYERVEDIC.name'), f.col('LOW_BACTERIA.name'),\
     f.col('COELIAC.name'), f.col('DIABETIC.name'), f.col('FREE_FROM_GLUTEN.name'), f.col('GLYCEMIC.name'),\
     f.col('GRAIN_FREE.name'), f.col('HALAL.name'), f.col('HGC.name'), f.col('HIGH_PROTEIN.name'),\
     f.col('KEHILLA.name'), f.col('KETOGENIC.name'), f.col('KOSHER.name'), f.col('LACTOSE_FREE.name'),\
     f.col('LOW_CALORIE.name'), f.col('LOW_PROTEIN.name'), f.col('LOW_SALT.name'), f.col('MACROBIOTIC.name'),\
     f.col('METABOLIC.name'), f.col('NON_VEG.name'), f.col('PALEO.name'),\
     f.col('PECETARIAN.name'), f.col('PLANT_BASED.name'), f.col('RAW_FOOD.name'), f.col('VEGAN.name'),\
     f.col('VEGETARIAN.name'), f.col('VEG_OVO.name'), f.col('WITHOUT_BEEF.name'), f.col('WITHOUT_PORK.name')))\
     .withColumn('clean_commas', f.regexp_replace(f.col('diet'), ' ,', ','))\
     .withColumn('lowercase', f.lower(concat_ws(',', 'clean_commas')))\
     .withColumn('no_trailing_space', regexp_replace(f.col('lowercase'), r'\s+$', ''))\
     .withColumnRenamed('no_trailing_space', 'pim_sentence')\
     .select('upc_key', 'krogerOwnedEcommerceDescription', 'gtinName', 'pim_sentence')

In [0]:
#Build sentence with upc, name and diet related meta data from Pinto data source
pinto_diets = pinto_prods.select(f.explode(f.col('upcs.standard')).alias('gtin_no'), f.col('dietList'))\
    .withColumn("gtin_no",expr("substring(gtin_no, 1, length(gtin_no)-1)"))\
    .select(f.col('gtin_no'), f.col('dietList.slug').alias('diet_slug'))
pinto_names = pinto_prods.select(f.explode(f.col('upcs.standard')).alias('gtin_no'), f.col('name'))\
    .withColumn("gtin_no",expr("substring(gtin_no, 1, length(gtin_no)-1)"))\
    .select('gtin_no', 'name').distinct()
pinto_data = pinto_names.join(pinto_diets, 'gtin_no', 'inner')

In [0]:
#Join Pinto and PIM data
pinto_pim = pinto_data.join(pim_diets, pim_diets.upc_key == pinto_data.gtin_no, 'outer')

from pyspark.sql.functions import when
pinto_pim = pinto_pim.withColumn("name", when(pinto_pim.name.isNull(), pinto_pim.krogerOwnedEcommerceDescription).otherwise(pinto_pim.name))

pinto_pim = pinto_pim.withColumn("krogerOwnedEcommerceDescription", when(pinto_pim.krogerOwnedEcommerceDescription.isNull(), pinto_pim.name).otherwise(pinto_pim.krogerOwnedEcommerceDescription))

In [0]:
# Clean data

sentences = pinto_pim.withColumn('diet_string', concat_ws(',  ', f.col('diet_slug')))\
    .withColumn('diet_string', f.lower(concat(f.col('diet_string'))))\
    .withColumn('diet_sentence', split(f.col('diet_string'), ', '))\
    .withColumn('pimto_sentence', concat_ws(',', f.col('diet_sentence'),f.col('pim_sentence')))\
    .withColumn('no_leading_comma', regexp_replace(f.col('pimto_sentence'), r'^\,+', ''))\
    .select('upc_key','name','gtin_no','no_leading_comma')
sentences = sentences.select(f.coalesce(sentences["upc_key"], sentences["gtin_no"]).alias('gtin_no'), f.col("name"),f.col('no_leading_comma').alias('diet_sentence'))
sentences = sentences.select(f.col("gtin_no"), f.col("name"),split(f.col("diet_sentence"),",").alias("diet_sentence_Arr"))

from pyspark.sql.functions import array_distinct
sentences = sentences.withColumn("dedup_diet_sentence_Arr", array_distinct("diet_sentence_Arr"))\
     .withColumnRenamed('dedup_diet_sentence_Arr', 'diet_sentence')\
     .withColumn("diet_sentence", concat_ws(",",f.col("diet_sentence")))\
     .select('gtin_no', 'name', 'diet_sentence')

In [0]:
# Create a "text" field that concatenates everything you want to encode(a string that has the sentence as well as gtin)
vector = sentences.select('diet_sentence', 'gtin_no', 'name')\
    .withColumn('sentence_string', concat_ws(',', f.col('diet_sentence')))\
    .withColumn('full_string', when(f.col('sentence_string') == '', concat(f.col('name'), f.lit('.'))).otherwise(concat(f.col('name'), f.lit(' is '), f.col('sentence_string'), f.lit('.'))))\
    .withColumn('diet_string', f.regexp_replace(f.col('full_string'), ',,', ','))\
    .select('gtin_no', 'diet_string')
vector = vector.join(pyear_purchased_upcs, pyear_purchased_upcs.upc == vector.gtin_no)
vector = vector.select('gtin_no', 'diet_string')

In [0]:
pandas_df = vector.toPandas()

In [0]:
# Specify the model directory on DBFS
model_dir = "/dbfs/dbfs/FileStore/users/s354840/pretrained_transformer_model" 

# These packages are required for delivery
from sentence_transformers import SentenceTransformer, util

# Loading the transformer model
model = SentenceTransformer(model_dir) 

In [0]:
# Extract "text" field into a list
sentence = pandas_df.diet_string.tolist() 

In [0]:
# Encode vectors from sentences
# You may need a GPU cluster to do this efficiently 3:01 p.m.
vectors = model.encode(sentence, normalize_embeddings=True) 


In [0]:
pandas_df['vector'] = vectors.tolist()

In [0]:
# Join together into output dataframe
import pandas as pd
output_df = spark.createDataFrame(pd.DataFrame({"gtin_no": pandas_df["gtin_no"].to_list(), "vector": pandas_df["vector"].to_list()}))
output_df.write.mode("overwrite").parquet('abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/Users/s354840/embedded_dimensions/last_year_product_vectors_diet_description/' + today + '/upc_vectors')